In [1]:
import re
import pandas as pd
from glob import glob
from tqdm import tqdm

# Dataset:

In [2]:
datasets_ = []
for dataset in ['keshmirian', 'greene', 'korner', 'oxford_utilitarianism_scale', 'cni']:
    df = pd.read_json(f'data/{dataset}/data.json')
    df['dataset'] = dataset
    datasets_.append(df)

datasets_info = pd.concat(datasets_)
datasets_info

,index,type,scenario,subject,dataset,name,violation
0,1,action,Emilia is negotiating with a powerful and dete...,Emilia’s,keshmirian,NaN,NaN
1,2,action,Azzo is the captain of a military submarine tr...,Azzo’s,keshmirian,NaN,NaN
2,3,action,Ann is the late-nightwatch nurse in a hospital...,Anna’s,keshmirian,NaN,NaN
3,4,action,A runaway trolley is heading down the tracks t...,Friedrich’s,keshmirian,NaN,NaN
4,5,action,Alex is the leader of a small group of soldier...,Alex’s,keshmirian,NaN,NaN
...,...,...,...,...,...,...,...
19,20,d6acinc,You are a doctor in an area that suffers from ...,your,cni,N/A,NaN
20,21,d3acinc,You are a member of a special police departmen...,your,cni,N/A,NaN
21,22,d4ininc,You are a doctor and are treating a seriously ...,your,cni,N/A,NaN
22,23,d5incon,You are the director of a hospital in a develo...,your,cni,N/A,NaN


In [3]:
NUM_ROUNDS = 6

# Single Agent:

In [4]:
def get_files(group_size, nob=False):
  if nob:
      files = glob(fr'runs/*/*/{group_size}n/*.jsonl')
  else:
      files = glob(fr'runs/*/*/{group_size}/*.jsonl')

  data = []
  for file in tqdm(files):
    try:
      match = re.match('runs/(.*)/(.*)/(.*)/(.*)_ob(.*)_(.*).jsonl', file)
      dataset = match.group(1)
      model = match.group(2)
      #   group_size_ = match.group(3).strip('n')
      example_index = match.group(4)
      ob = match.group(5)
      rep = match.group(6)

      df = pd.read_json(file, lines=True)
      df = df[[col for col in df.columns]]
      df['dataset'] = dataset
      df['model'] = model
      df['group_size'] = group_size
      df['example_index'] = int(example_index)
      df['ob'] = ob
      df['rep'] = int(rep)
      df['step'] = -1  if group_size ==1 else df.index//group_size
      data.append(df)
    except Exception as e:
      print(f"Error reading file {file}: {e}")

  return pd.concat(data)

In [5]:
single = get_files(1, nob=True)

100%|██████████| 51912/51912 [02:33<00:00, 338.90it/s]


In [6]:
print('[SINGLE] Cleaning opinions ...')

single_ = single.copy()
patterns_to_remove = [
    (r'\(.*?\)', '', True),
    (r'\s*\([^)]*\)', '', True),
    (r'\\boxed\{(\d+)\}', r'\1', True),
    ('**', '', False),
    (' out of 7', '', False),
    ('.out.of.', '', False),
    ('=out=of=7', '', False), 
    ('\tout\tof\t', '', False),
    ('=', '', False),
    ('/7', '', False),
    ('/ 7', '', False),
    (', 7', '', False),
    ('"', '', False),
    ("moral acceptability score:", '', False),
    ("score:", '', False),
    ("final answer", '', False),
    ("final score: ", '', False), 
    ("final rating: ", '', False),
    ("agreement rating: ", '', False),
    ("– mostly unacceptable.", '', False),
    ("– completely unacceptable.", '', False),
    (": completely unacceptable", '', False),
    ("– somewhat morally unacceptable", '', False),
    ("final rating:", '', False),
    ("final answer", '', False),
    ("- strongly disagree", '', False),
    ("– strongly disagree", '', False),
    ("– completely acceptable", '', False),
    ("–completely acceptable", '', False),
    ("—completely acceptable", '', False),
    ("— completely unacceptable", '', False),
    ("– completely unacceptable", '', False),
    ("– somewhat morally acceptable", '', False),
    (": strongly acceptable", '', False),
    ("final agreement rating:", '', False),
    ("score:", '', False),
    ("– somewhat disagree", '', False),
    ("– disagree", '', False),
    ("- morally acceptable", '', False),
    ("the answer is", '', False),
    ("— strongly morally acceptable", '', False)
]

single_['opinion'] = single_['opinion'].astype(str).str.lower()
for pattern, repl, is_regex in patterns_to_remove:
    single_['opinion'] = single_['opinion'].str.replace(pattern, repl, regex=is_regex)

single_['opinion'] = single_['opinion'].str.strip().str.strip('.')

[SINGLE] Cleaning opinions ...


In [ ]:
single_['opinion'].value_counts()

In [8]:
single = single_

In [9]:
single = single[~single['model'].isin(['gemma3', 'o4-mini'])]

In [10]:
single['model'].unique()

array(['gemma3:27b', 'gpt-4.1', 'llama3.3', 'qwen2.5:32b-instruct',
       'qwen3:32b', 'qwq'], dtype=object)

In [11]:
single['type'] = single.apply(lambda x: datasets_info[(datasets_info['dataset'] == x['dataset']) & (datasets_info['index'] == x['example_index'])]['type'].values[0], axis=1)

/tmp/ipykernel_2674485/3183118866.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single['type'] = single.apply(lambda x: datasets_info[(datasets_info['dataset'] == x['dataset']) & (datasets_info['index'] == x['example_index'])]['type'].values[0], axis=1)


In [12]:
print("[ SINGLE ]")
example_counts = single.groupby('dataset')['model'].value_counts().unstack(fill_value=0).sort_index(level='dataset') 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display((example_counts/50).astype(int).T)

[ SINGLE ]


dataset,cni,greene,keshmirian,korner,oxford_utilitarianism_scale
model,,,,,
gemma3:27b,24,64,16,38,9
gpt-4.1,24,64,16,38,9
llama3.3,24,64,16,38,9
qwen2.5:32b-instruct,24,64,16,38,9
qwen3:32b,24,64,16,38,9
qwq,24,64,16,38,9


# Dyadic Groups:

In [13]:
group_size_dyad = 2
dyad = get_files(group_size_dyad, nob=True)

100%|██████████| 2820/2820 [00:08<00:00, 351.92it/s]


In [14]:
num_replies_dyad = NUM_ROUNDS*group_size_dyad + 2*group_size_dyad

In [15]:
print('[DYAD] Cleaning opinions ...')
dyad_ = dyad.copy()
patterns_to_remove = [
    (r'\(.*?\)', '', True),
    (r'\s*\([^)]*\)', '', True),
    (r'\\boxed\{(\d+)\}', r'\1', True),
    ('**', '', False),
    ('#', '', False),
    ('```text', '', False),
    ('```', '', False),
    (' out of 7', '', False),
    ('.out.of.', '', False),
    ('=out=of=7', '', False),
    ('\tout\tof\t', '', False),
    ('=', '', False),
    ('/7', '', False),
    ('/ 7', '', False),
    (', 7', '', False),
    ('"', '', False),
    ("moral acceptability score:", '', False),
    ("score:", '', False),
    ("final answer", '', False),
    ("final score: ", '', False),
    ("final rating: ", '', False),
    ("agreement rating: ", '', False),
    ("– mostly unacceptable.", '', False),
    ("– completely unacceptable.", '', False),
    (": completely unacceptable", '', False),
    ("– somewhat morally unacceptable", '', False),
    ("final rating:", '', False),
    ("– strongly disagree", '', False),
    ("- strongly disagree", '', False),
    ("– completely acceptable", '', False),
    ("–completely acceptable", '', False),
    ("—completely acceptable", '', False),
    ("— completely unacceptable", '', False),
    ("– completely unacceptable", '', False),
    ("- completely unacceptable", '', False),
    ("final agreement rating:", '', False)
]

dyad_['opinion'] = dyad_['opinion'].astype(str).str.lower()
for pattern, repl, is_regex in patterns_to_remove:
    dyad_['opinion'] = dyad_['opinion'].str.replace(pattern, repl, regex=is_regex)

dyad_['opinion'] = dyad_['opinion'].str.strip().str.strip('.')

[DYAD] Cleaning opinions ...


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    nan_mask = dyad_['opinion'].isna()
    conversion_mask = pd.to_numeric(dyad_['opinion'], errors='coerce').isna() & ~nan_mask
    error_mask = conversion_mask
    error_examples = dyad_[error_mask]
    display(error_examples['opinion'].value_counts())

In [17]:
dyad = dyad_

In [18]:
dyad = dyad[~dyad['model'].isin(['gemma3', 'o4-mini'])]

In [19]:
dyad['model'].unique()

array(['gemma3:27b', 'gpt-4.1', 'llama3.3', 'qwen2.5:32b-instruct',
       'qwen3:32b', 'qwq'], dtype=object)

In [20]:
dyad['type'] = dyad.apply(lambda x: datasets_info[(datasets_info['dataset'] == x['dataset']) & (datasets_info['index'] == x['example_index'])]['type'].values[0], axis=1)

/tmp/ipykernel_2674485/3497721238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dyad['type'] = dyad.apply(lambda x: datasets_info[(datasets_info['dataset'] == x['dataset']) & (datasets_info['index'] == x['example_index'])]['type'].values[0], axis=1)


In [21]:
print("[ DYAD ]")
example_counts = dyad.groupby('dataset')['model'].value_counts().unstack(fill_value=0).sort_index(level='dataset') 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display((example_counts/num_replies_dyad/3).astype(int).T)

[ DYAD ]


dataset,cni,greene,keshmirian,korner,oxford_utilitarianism_scale
model,,,,,
gemma3:27b,24,64,16,38,9
gpt-4.1,24,64,16,38,9
llama3.3,24,64,16,38,9
qwen2.5:32b-instruct,24,64,16,38,9
qwen3:32b,24,64,16,38,9
qwq,24,64,16,38,9


# Triad Groups

In [22]:
group_size_traid = 3
triad = get_files(group_size_traid, nob=True)

100%|██████████| 2901/2901 [00:09<00:00, 319.30it/s]


In [23]:
num_replies_triad = NUM_ROUNDS*group_size_traid + 2*group_size_traid

In [24]:
print('[TRIAD] Cleaning opinions ...')
triad_ = triad.copy()
patterns_to_remove = [
    (r'\(.*?\)', '', True), 
    (r'\s*\([^)]*\)', '', True), 
    (r'\\boxed\{(\d+)\}', r'\1', True),
    ('**', '', False),
    (' out of 7', '', False),
    ('.out.of.', '', False), 
    ('=out=of=7', '', False),
    ('\tout\tof\t', '', False),
    ('=', '', False),
    ('/7', '', False),
    ('/ 7', '', False),
    (', 7', '', False),
    ('"', '', False),
    ("moral acceptability score:", '', False),
    ("score:", '', False),
    ("final answer", '', False), 
    ("final score: ", '', False),
    ("final rating: ", '', False),
    ("agreement rating: ", '', False),
    ("– mostly unacceptable.", '', False),
    ("– completely unacceptable.", '', False),
    (": completely unacceptable", '', False),
    ("– somewhat morally unacceptable", '', False),
    ("final rating:", '', False),
    ("final answer", '', False),
    ("- strongly disagree", '', False),
    ("– strongly disagree", '', False),
    ("– completely acceptable", '', False),
    ("–completely acceptable", '', False),
    ("—completely acceptable", '', False),
    ("— completely unacceptable", '', False),
    ("– completely unacceptable", '', False),
    ("– somewhat morally acceptable", '', False),
    (": strongly acceptable", '', False),
    ("final agreement rating:", '', False),
    ("score:", '', False),
    ("– somewhat disagree", '', False),
    ("– disagree", '', False),
    ("- morally acceptable", '', False),
    ("the answer is", '', False),
    ("— strongly morally acceptable", '', False)
]

triad_['opinion'] = triad_['opinion'].astype(str).str.lower()
for pattern, repl, is_regex in patterns_to_remove:
    triad_['opinion'] = triad_['opinion'].str.replace(pattern, repl, regex=is_regex)

triad_['opinion'] = triad_['opinion'].str.strip().str.strip('.')

[TRIAD] Cleaning opinions ...


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    nan_mask = triad_['opinion'].isna()
    conversion_mask = pd.to_numeric(triad_['opinion'], errors='coerce').isna() & ~nan_mask
    error_mask = conversion_mask 
    error_examples = triad_[error_mask]
    display(error_examples['opinion'].value_counts())

In [26]:
triad = triad_

In [27]:
triad = triad[~triad['model'].isin(['gemma3', 'o4-mini'])]

In [28]:
triad['model'].unique()

array(['gemma3:27b', 'gpt-4.1', 'llama3.3', 'qwen2.5:32b-instruct',
       'qwen3:32b', 'qwq'], dtype=object)

In [ ]:
triad['type'] = triad.apply(lambda x: datasets_info[(datasets_info['dataset'] == x['dataset']) & (datasets_info['index'] == x['example_index'])]['type'].values[0], axis=1)

In [30]:
print("[ TRIAD ]")
example_counts = triad.groupby('dataset')['model'].value_counts().unstack(fill_value=0).sort_index(level='dataset') 
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display((example_counts/num_replies_triad/3).astype(int).T)

[ TRIAD ]


dataset,cni,greene,keshmirian,korner,oxford_utilitarianism_scale
model,,,,,
gemma3:27b,24,64,16,38,9
gpt-4.1,24,64,16,38,9
llama3.3,24,64,16,38,9
qwen2.5:32b-instruct,24,64,16,38,9
qwen3:32b,24,64,16,38,9
qwq,24,64,16,38,9


In [31]:
single.to_csv(f'runs/single.csv', index=False)
single_no_message = single.drop(['message', 'reasoning'], axis=1)
single_no_message.to_csv('runs/single_no_message.csv', index=False)

dyad.to_csv(f'runs/nob_2.csv', index=False)
dyad_no_message = dyad.drop(['message', 'reasoning'], axis=1)
dyad_no_message.to_csv(f'runs/nob_2_no_message.csv', index=False)

triad.to_csv(f'runs/nob_3.csv', index=False)
triad_no_message = triad.drop(['message', 'reasoning'], axis=1)
triad_no_message.to_csv(f'runs/nob_3_no_message.csv', index=False)